In [7]:
import altair as alt
import pandas as pd

# norms
# mean sj: (1,1,0,0,0,0,1,1)
# nice sj: (1,1,0,1,0,1,1,1)

norms_b = ["(1,1,0,0,0,0,1,1)", "(1,1,0,1,0,1,1,1)", "(0,0,0,0,1,1,1,1)", "(0,1,0,1,1,1,1,1)",
          "(1,1,0,0,1,1,1,1)", "(1,1,0,1,1,1,1,1)", "(0,0,0,0,0,0,1,1)", "(0,1,0,1,0,1,1,1)"]

norms_n = ["SJ normal", "SJ nice", "IS normal", "IS nice", "SS normal", "SS nice", "SH normal", "SH normal"]

columns = ["norm", "z", "g", "mu", "chi", "eps", "pdx", "acr", "mAllD", "nAllD", "mpDisc", "npDisc", "mDisc", "nDisc", "mAllC", "nAllC"]

data: pd.DataFrame = pd.read_csv("results.txt", sep="\t", header=None, names=columns, index_col=False)

data.eps = data.eps / data.z

data.norm = data.norm.replace(norms_b, norms_n)

def filter(frame: pd.DataFrame, feature: str, condition):
    return frame[frame[feature]==condition]
    

In [8]:
data

,norm,z,g,mu,chi,eps,pdx,acr,mAllD,nAllD,mpDisc,npDisc,mDisc,nDisc,mAllC,nAllC
0,IS normal,50,5000,0.1,0.1,0.00002,True,21.680555,0.96,0.0,0.04,0.0,0.0,0.0,0.0,0.0
1,SS normal,50,5000,0.1,0.1,0.00002,True,55.934209,0.00,1.0,0.00,0.0,0.0,0.0,0.0,0.0
2,SJ normal,50,5000,0.1,0.1,0.00002,True,52.573041,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,IS normal,50,5000,0.1,0.1,0.00002,True,5.196343,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,SS normal,50,5000,0.1,0.1,0.00002,True,30.105799,0.00,0.0,1.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,SJ nice,50,5000,0.1,0.1,0.75000,True,8.793365,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2753,SS nice,50,5000,0.1,0.1,0.75000,True,4.324573,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2754,SS nice,50,5000,0.1,0.1,0.20000,True,6.497721,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2755,IS nice,50,5000,0.1,0.1,0.75000,True,6.381351,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [9]:
filtered: pd.DataFrame = filter(data, "z", 50)

In [25]:
alt.Chart(filtered).mark_line(point=True).encode(
    x=alt.X('eps:Q', scale=alt.Scale(type="log")),
    y='mean(acr):Q',
    color='norm:N'
)

alt.Chart(...)

In [23]:
new_data = filtered[filtered.norm == "SJ normal"]
new_cols = ["eps", "strat", "acr"]
new_df = pd.DataFrame(columns=new_cols)
strats = ["mAllD", "nAllD", "mpDisc", "npDisc", "mDisc", "nDisc", "mAllC", "nAllC"]

for epsilon in new_data.eps.unique():
    print("----- epsilon: ", epsilon, "-------" )
    for strat in strats:
        mean: float = new_data[strat][new_data.eps==epsilon].mean()
        print(epsilon, strat, mean)
        new_df.loc[len(new_df)] = [epsilon, strat, mean]

----- epsilon:  2e-05 -------
2e-05 mAllD 0.24
2e-05 nAllD 0.1052
2e-05 mpDisc 0.0876
2e-05 npDisc 0.09960000000000001
2e-05 mDisc 0.17200000000000004
2e-05 nDisc 0.2048
2e-05 mAllC 0.039200000000000006
2e-05 nAllC 0.0516
----- epsilon:  0.0002 -------
0.0002 mAllD 0.0812
0.0002 nAllD 0.1392
0.0002 mpDisc 0.25880000000000003
0.0002 npDisc 0.15480000000000002
0.0002 mDisc 0.1368
0.0002 nDisc 0.19400000000000003
0.0002 mAllC 0.0104
0.0002 nAllC 0.024800000000000003
----- epsilon:  0.002 -------
0.002 mAllD 0.0592
0.002 nAllD 0.09879999999999999
0.002 mpDisc 0.2648
0.002 npDisc 0.15280000000000002
0.002 mDisc 0.174
0.002 nDisc 0.23879999999999998
0.002 mAllC 0.0012
0.002 nAllC 0.0104
----- epsilon:  0.02 -------
0.02 mAllD 0.0
0.02 nAllD 0.0004
0.02 mpDisc 0.2336
0.02 npDisc 0.26039999999999996
0.02 mDisc 0.2568
0.02 nDisc 0.2396
0.02 mAllC 0.0076
0.02 nAllC 0.0016
----- epsilon:  0.2 -------
0.2 mAllD 0.0102
0.2 nAllD 0.02
0.2 mpDisc 0.2566
0.2 npDisc 0.24220000000000003
0.2 mDisc 0.2414

In [24]:
alt.Chart(new_df).mark_line(point=True).encode(
    x=alt.X('eps:Q', scale=alt.Scale(type="log")),
    y = 'acr',
    color = 'strat'
)

alt.Chart(...)